In [1]:
from src.environment._ENV_capture import Board
from src.agent._AGENT_capture import Agent
from src.learn._q_learning_capture import Q_learning

# Capture chess
### Q-learning

In [2]:
board = Board()
agent = Agent(network='conv',gamma=0.1,lr=0.07)
R = Q_learning(agent,board)

In [3]:
game = R.learn(100)

iter 0


UnimplementedError: Graph execution error:

Detected at node 'model/conv2d/Conv2D' defined at (most recent call last):
    File "/Users/valeriainsogna/anaconda3/envs/RL/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/valeriainsogna/anaconda3/envs/RL/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/valeriainsogna/anaconda3/envs/RL/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/valeriainsogna/anaconda3/envs/RL/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/Users/valeriainsogna/anaconda3/envs/RL/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 728, in start
      self.io_loop.start()
    File "/Users/valeriainsogna/anaconda3/envs/RL/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 195, in start
      